+ 读取全部数据；
+ 对于 model 项为空的进行填充
+ 根据时间切分时间窗口（尝试先切分窗口，再进行汇总）

读取数据

In [3]:
import pandas  as pd 
import numpy as np
#定义读取数据的函数
def read_action_data(address):
    reader = pd.read_csv(address, sep = ',', iterator = True,parse_dates=['time'])
    chunks = []
    loop = True
    i = 1
    while loop:
        try:
            chunk = reader.get_chunk(30000000)
            chunks.append(chunk)
            print (i)
            i += 1
        except StopIteration:
            loop = False
            print ('Iteration is stopped.')
    data = pd.concat(chunks, ignore_index = True)
    print("DONE")
    return data 


In [4]:
# 读取2月份数据测试
address02="~/Desktop/dataAnalysis/JDATA/JData/JData_Action_201602.csv"
# address03="~/Desktop/dataAnalysis/JDATA/JData/JData_Action_201603.csv"
# address04="~/Desktop/dataAnalysis/JDATA/JData/JData_Action_201604.csv"
df_02=read_action_data(address02)
# df_03=read_data(address03)
# df_04=read_data(address04)
# df_action=pd.concat([df_02,df_03,df_04],axis=0,ignore_index=True)
# print(df_action.shape)
print(df_02.shape)
df_02.head()

1
Iteration is stopped.
DONE
(11485424, 7)


,user_id,sku_id,time,model_id,type,cate,brand
0,266079.0,138778,2016-01-31 23:59:02,NaN,1,8,403
1,266079.0,138778,2016-01-31 23:59:03,0.0,6,8,403
2,200719.0,61226,2016-01-31 23:59:07,NaN,1,8,30
3,200719.0,61226,2016-01-31 23:59:08,0.0,6,8,30
4,263587.0,72348,2016-01-31 23:59:08,NaN,1,5,159


In [4]:
# check the time column dtype
df_02[["time"]].dtypes

time    datetime64[ns]
dtype: object

In [5]:
# fill and drop the model_id column nan value
df2=df_02.dropna(how='any')
df2.head()

,user_id,sku_id,time,model_id,type,cate,brand
1,266079.0,138778,2016-01-31 23:59:03,0.0,6,8,403
3,200719.0,61226,2016-01-31 23:59:08,0.0,6,8,30
6,296130.0,103126,2016-01-31 23:59:11,0.0,6,4,174
7,296130.0,103126,2016-01-31 23:59:11,0.0,6,4,174
8,217892.0,137328,2016-01-31 23:59:14,111.0,6,6,159


In [5]:
# 时间窗口的选择

df_buy=df_02.ix[df_02["type"]==4,:]
print (df_buy.shape)

# df_buy.groupby(['user_id','sku_id']).count()
# df_02.ix[(df_02['time']<df_buy['time']) & (df_02['user_id']==df_buy['user_id']) & (df_02['sku_id']==df_buy['sku_id']),:]

df_buy.head()

(11485, 7)


,user_id,sku_id,time,model_id,type,cate,brand
351,269365.0,166345,2016-02-01 00:01:00,NaN,4,9,306
649,235443.0,36692,2016-02-01 00:02:00,NaN,4,4,174
980,247689.0,9112,2016-02-01 00:03:00,NaN,4,5,78
1719,273959.0,102034,2016-02-01 00:05:00,NaN,4,5,78
2153,226791.0,163550,2016-02-01 00:06:00,NaN,4,4,306


In [7]:
# for i in df_buy["user_id"].values:
#     time_end=df_buy.ix[df_buy["user_id"]==i,'time']
#     time_start=time_end-15

# import datetime 
# # df_buy["time"]
# print(df_02.index)
# # df_02=df_02.set_index([df_02.index,df_02['time']],drop=False)
# df02=df_02.set_index([df_02['time']],drop=False)

In [7]:
df02.head()

,user_id,sku_id,time,model_id,type,cate,brand
time,,,,,,,
2016-01-31 23:59:02,266079.0,138778,2016-01-31 23:59:02,NaN,1,8,403
2016-01-31 23:59:03,266079.0,138778,2016-01-31 23:59:03,0.0,6,8,403
2016-01-31 23:59:07,200719.0,61226,2016-01-31 23:59:07,NaN,1,8,30
2016-01-31 23:59:08,200719.0,61226,2016-01-31 23:59:08,0.0,6,8,30
2016-01-31 23:59:08,263587.0,72348,2016-01-31 23:59:08,NaN,1,5,159


In [6]:
import datetime
# start=datetime.datetime.strptime('2016-02-01','%Y-%m-%d')
# end=datetime.datetime.strptime('2016-02-03','%Y-%m-%d')
# print(start,end)
# df02['time'].between_time('2016-02-01 00:00:00','2016-02-03 00:00:00')
delta_time=datetime.timedelta(days=-7)
df_buy["start_time"]=df_buy['time']+delta_time
# df_buy.head()
# mask = (df_buy['start_time'] >= '2016-02-01') & (df_buy['time'] <= '2016-02-08')
# df_buy.loc[df_buy['start_time'] >= '2016-02-01'].shape[0]

/Users/xiaolin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
# mask = (df_buy['start_time'] >= '2016-02-01') & (df_buy['time'] <= '2016-02-08')
# df_buy.loc[df_buy['start_time'] >= '2016-02-01'].shape[0]
df_buy

,user_id,sku_id,time,model_id,type,cate,brand,sart_time,start_time
351,269365.0,166345,2016-02-01 00:01:00,NaN,4,9,306,2016-01-25 00:01:00,2016-01-25 00:01:00
649,235443.0,36692,2016-02-01 00:02:00,NaN,4,4,174,2016-01-25 00:02:00,2016-01-25 00:02:00
980,247689.0,9112,2016-02-01 00:03:00,NaN,4,5,78,2016-01-25 00:03:00,2016-01-25 00:03:00
1719,273959.0,102034,2016-02-01 00:05:00,NaN,4,5,78,2016-01-25 00:05:00,2016-01-25 00:05:00
2153,226791.0,163550,2016-02-01 00:06:00,NaN,4,4,306,2016-01-25 00:06:00,2016-01-25 00:06:00
2877,301962.0,125188,2016-02-01 00:08:00,NaN,4,5,479,2016-01-25 00:08:00,2016-01-25 00:08:00
2881,246290.0,125188,2016-02-01 00:08:00,NaN,4,5,479,2016-01-25 00:08:00,2016-01-25 00:08:00
3210,280086.0,79616,2016-02-01 00:09:00,NaN,4,4,36,2016-01-25 00:09:00,2016-01-25 00:09:00
3211,280086.0,131300,2016-02-01 00:09:00,NaN,4,8,693,2016-01-25 00:09:00,2016-01-25 00:09:00
3213,280086.0,108328,2016-02-01 00:09:00,NaN,4,5,800,2016-01-25 00:09:00,2016-01-25 00:09:00


In [7]:
# mask = (df_buy['start_time'] >= '2016-02-01') & (df_buy['time'] <= '2016-02-08')
df_buy02=df_buy.loc[df_buy['start_time'] >= '2016-02-01']
df_buy02.head()

,user_id,sku_id,time,model_id,type,cate,brand,start_time
1944268,232340.0,20330,2016-02-08 00:01:00,NaN,4,5,306,2016-02-01 00:01:00
1946099,211031.0,39139,2016-02-08 00:14:00,NaN,4,7,306,2016-02-01 00:14:00
1947851,233827.0,33955,2016-02-08 00:24:00,NaN,4,5,375,2016-02-01 00:24:00
1947852,233827.0,64160,2016-02-08 00:24:00,NaN,4,6,232,2016-02-01 00:24:00
1949891,245130.0,76996,2016-02-08 00:36:00,NaN,4,7,489,2016-02-01 00:36:00


In [46]:
df_02.ix[(df_02["user_id"]==232340)&(df_02["sku_id"]==20330),:]

,user_id,sku_id,time,model_id,type,cate,brand
1217300,232340.0,20330,2016-02-04 19:45:36,NaN,1,5,306
1217301,232340.0,20330,2016-02-04 19:45:36,0.0,6,5,306
1217302,232340.0,20330,2016-02-04 19:45:36,216.0,6,5,306
1217303,232340.0,20330,2016-02-04 19:45:36,217.0,6,5,306
1217304,232340.0,20330,2016-02-04 19:45:36,NaN,1,5,306
1217429,232340.0,20330,2016-02-04 19:46:07,29.0,6,5,306
1217542,232340.0,20330,2016-02-04 19:46:38,NaN,1,5,306
1217543,232340.0,20330,2016-02-04 19:46:38,26.0,6,5,306
1217544,232340.0,20330,2016-02-04 19:46:38,NaN,1,5,306
1217642,232340.0,20330,2016-02-04 19:47:09,25.0,6,5,306


In [8]:
#合并开始时间到全量2月份数据

df_02_new=pd.merge(df_02,df_buy02[["user_id","sku_id","start_time"]],how="left",on=["user_id","sku_id"]).drop_duplicates()
df_02_new.head()

,user_id,sku_id,time,model_id,type,cate,brand,start_time
0,266079.0,138778,2016-01-31 23:59:02,NaN,1,8,403,NaT
1,266079.0,138778,2016-01-31 23:59:03,0.0,6,8,403,NaT
2,200719.0,61226,2016-01-31 23:59:07,NaN,1,8,30,NaT
3,200719.0,61226,2016-01-31 23:59:08,0.0,6,8,30,NaT
4,263587.0,72348,2016-01-31 23:59:08,NaN,1,5,159,NaT


In [63]:
print(df_02_new.shape)

(8742163, 8)


In [61]:
df_02_buy_action=df_02_new.loc[df_02_new['time']>=df_02_new['start_time']].drop_duplicates() #去重
df_02_buy_action.head()

,user_id,sku_id,time,model_id,type,cate,brand,start_time
106300,211031.0,39139,2016-02-01 10:57:41,221.0,6,7,306,2016-02-01 00:14:00
106301,211031.0,39139,2016-02-01 10:57:41,NaN,1,7,306,2016-02-01 00:14:00
106303,211031.0,39139,2016-02-01 10:57:41,217.0,6,7,306,2016-02-01 00:14:00
106306,211031.0,39139,2016-02-01 10:57:41,216.0,6,7,306,2016-02-01 00:14:00
106308,211031.0,39139,2016-02-01 10:57:41,0.0,6,7,306,2016-02-01 00:14:00


In [52]:
# df_not_buy=df_02.ix[df_02["type"]!=4,:]
# print (df_not_buy.shape)
# delta_time=datetime.timedelta(days=-7)
# df_not_buy["start_time"]=df_not_buy['time']+delta_time

In [13]:

df_buy=df_02.ix[df_02["type"]==4,:]
print (df_buy.shape)
# dropfunc=lambda row:row[["user_id","sku_id"]] not in df_buy[["user_id","sku_id"]]
# df_02_not_buy= df_02[df_02.apply(dropfunc, axis=1)]
df_02_not_buy=df_02[~df_02[["user_id","sku_id","model_id","type","cate","brand"]].isin(df_buy[["user_id","sku_id"]])]

(11485, 7)


In [14]:
df_02_not_buy

,user_id,sku_id,time,model_id,type,cate,brand
0,266079.0,138778.0,NaT,NaN,1,8,403
1,266079.0,138778.0,NaT,0.0,6,8,403
2,200719.0,61226.0,NaT,NaN,1,8,30
3,200719.0,61226.0,NaT,0.0,6,8,30
4,263587.0,72348.0,NaT,NaN,1,5,159
5,296130.0,103126.0,NaT,NaN,1,4,174
6,296130.0,103126.0,NaT,0.0,6,4,174
7,296130.0,103126.0,NaT,0.0,6,4,174
8,217892.0,137328.0,NaT,111.0,6,6,159
9,283139.0,4732.0,NaT,NaN,1,6,375


In [ ]:
df_02_not_buy_new=pd.merge(df_02_not_buy[["user_id","sku_id","model_id","type","cate","brand"]],df_02[["user_id","sku_id","time"]],how="left",on=["user_id","sku_id"]).drop_duplicates()
df_02_not_buy_new